In [1]:
import json
import pandas as pd

f = open('/data/shith/ml-1m/ratings.dat', 'r')
data = f.readlines()
f = open('/data/shith/ml-1m/movies.dat', 'r', encoding='ISO-8859-1')
movies = f.readlines()
f = open('/data/shith/ml-1m/users.dat', 'r')
users = f.readlines()

movie_names = [_.split('::')[1] for _ in movies]  # Movie names
user_ids = [_.split('::')[0] for _ in users]      # User IDs
movie_ids = [_.split('::')[0] for _ in movies]     # Movie IDs


In [2]:
movie_id_to_name = {}
for movie_id, movie_name in zip(movie_ids, movie_names):
    movie_id_to_name[int(movie_id)] = movie_name
print(movie_id_to_name[3948])

Meet the Parents (2000)


In [3]:
interaction_dicts = dict()  
for line in data:
    user_id, movie_id, rating, timestamp = line.strip().split('::')
    if user_id not in interaction_dicts:
        interaction_dicts[user_id] = {
            'movie_id': [],
            'rating': [],
            'timestamp': [],
        }
    interaction_dicts[user_id]['movie_id'].append(movie_id)
    interaction_dicts[user_id]['rating'].append(int(int(rating) > 3))
    interaction_dicts[user_id]['timestamp'].append(int(timestamp))


In [4]:
sequential_interaction_list = []
seq_len = 10

for user_id in interaction_dicts:
    temp = zip(interaction_dicts[user_id]['movie_id'], interaction_dicts[user_id]['rating'], interaction_dicts[user_id]['timestamp'])
    temp = sorted(temp, key=lambda x: x[2])
    _,__,time = zip(*temp)
    # print(time)
    time_list = []
    for s in time_list:
        time_list.append(int(s))
    is_increasing = all(time_list[i] <= time_list[i + 1] for i in range(len(time_list) - 1))
    if not is_increasing:
        print('not increasing')
        break
    result = zip(*temp)
    interaction_dicts[user_id]['movie_id'], interaction_dicts[user_id]['rating'], interaction_dicts[user_id]['timestamp'] = [list(_) for _ in result]
    for i in range(10, len(interaction_dicts[user_id]['movie_id'])):
        sequential_interaction_list.append(
            [user_id, interaction_dicts[user_id]['timestamp'][i-seq_len:i], interaction_dicts[user_id]['movie_id'][i-seq_len:i], interaction_dicts[user_id]['rating'][i-seq_len:i], interaction_dicts[user_id]['movie_id'][i], interaction_dicts[user_id]['rating'][i], interaction_dicts[user_id]['timestamp'][i]]
        )

In [5]:
sequential_interaction_list = sorted(sequential_interaction_list, key=lambda x: int(x[6]))
times = [int(sequential_interaction[6]) for sequential_interaction in sequential_interaction_list]
is_increasing = (times==sorted(times))
print(is_increasing)
print(sequential_interaction_list[0])

True
['6040', [956703932, 956703954, 956703954, 956703977, 956703977, 956704056, 956704056, 956704056, 956704056, 956704056], ['858', '593', '2384', '1961', '2019', '573', '1419', '213', '3111', '3505'], [1, 1, 1, 1, 1, 1, 0, 1, 1, 1], '1734', 0, 956704081]


In [6]:
if len(sequential_interaction_list) >= 50000:
    sequential_interaction_list = sequential_interaction_list[-200000:]

In [7]:
# Find the earliest and latest timestamps in sequential_interaction_list
from datetime import datetime

timestamps = [int(item[-1]) for item in sequential_interaction_list]  # Extract timestamps from the sequential_interaction_list

# Convert timestamps from seconds to datetime objects
timestamps = [datetime.fromtimestamp(timestamp) for timestamp in timestamps]

# Find the earliest and latest timestamps
earliest_timestamp = min(timestamps)
latest_timestamp = max(timestamps)

# Print the results in a human-readable format
print("Earliest Timestamp:", earliest_timestamp.strftime('%Y-%m-%d %H:%M:%S'))
print("Latest Timestamp:", latest_timestamp.strftime('%Y-%m-%d %H:%M:%S'))


Earliest Timestamp: 2000-12-02 00:57:15
Latest Timestamp: 2003-03-01 01:49:50


In [8]:
from datetime import datetime

total_length = len(sequential_interaction_list)
chunk_size = total_length // 20

for i in range(20):
    start_index = i * chunk_size
    end_index = start_index + chunk_size if i < 19 else total_length
    chunk = sequential_interaction_list[start_index:end_index]
    
    timestamps = [int(item[-1]) for item in chunk]
    timestamps = [datetime.fromtimestamp(timestamp) for timestamp in timestamps]
    
    earliest_timestamp = min(timestamps)
    latest_timestamp = max(timestamps)
    
    print(f"period {i} - Earliest Timestamp: {earliest_timestamp.strftime('%Y-%m-%d %H:%M:%S')}, Latest Timestamp: {latest_timestamp.strftime('%Y-%m-%d %H:%M:%S')}")


period 0 - Earliest Timestamp: 2000-12-02 00:57:15, Latest Timestamp: 2000-12-03 09:44:24
period 1 - Earliest Timestamp: 2000-12-03 09:44:24, Latest Timestamp: 2000-12-06 06:11:22
period 2 - Earliest Timestamp: 2000-12-06 06:11:22, Latest Timestamp: 2000-12-08 00:23:57
period 3 - Earliest Timestamp: 2000-12-08 00:24:00, Latest Timestamp: 2000-12-08 14:54:36
period 4 - Earliest Timestamp: 2000-12-08 14:54:36, Latest Timestamp: 2000-12-09 15:01:20
period 5 - Earliest Timestamp: 2000-12-09 15:01:20, Latest Timestamp: 2000-12-12 08:49:01
period 6 - Earliest Timestamp: 2000-12-12 08:49:01, Latest Timestamp: 2000-12-15 06:23:09
period 7 - Earliest Timestamp: 2000-12-15 06:23:09, Latest Timestamp: 2000-12-19 07:45:14
period 8 - Earliest Timestamp: 2000-12-19 07:45:30, Latest Timestamp: 2000-12-23 08:21:59
period 9 - Earliest Timestamp: 2000-12-23 08:21:59, Latest Timestamp: 2000-12-30 04:43:18
period 10 - Earliest Timestamp: 2000-12-30 04:45:29, Latest Timestamp: 2001-01-07 07:08:06
period 11

In [ ]:
import csv
# save the csv file for baselines
num_parts = 20
part_size = len(sequential_interaction_list) // num_parts
for j in range(0,num_parts):
    with open(f'./all{j}.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['user_id', 'history_timestamp', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'])
        start_index = j * part_size
        end_index = start_index + part_size
        part_data = sequential_interaction_list[start_index:end_index]
        writer.writerows(part_data)


In [15]:

def csv_to_csv(input_path, output_num):
    data = pd.read_csv(input_path)
    # import random
    train_data = data[:int(0.9 * data.shape[0])]
    val_data = data[int(0.9 * data.shape[0]):]
    test_data = data[:]
    train_data.to_csv(f'./train{output_num}.csv')
    val_data.to_csv(f'./val{output_num}.csv')
    test_data.to_csv(f'./test{output_num}.csv')
for j in range(20):
    csv_to_csv(f'./all{j}.csv', j)


In [16]:
def csv_to_json(input_path, output_path):
    data = pd.read_csv(input_path)
    part_data = data.copy()
    json_list = []
    print(part_data.max())
    part_data.sort_values(by = ['user_id', 'timestamp'], ascending= True,inplace=True)
    for index, row in part_data.iterrows():
        row['history_movie_id'] = eval(row['history_movie_id'])
        row['history_rating'] = eval(row['history_rating'])
        L = len(row['history_movie_id'])
        preference = []
        unpreference = []
        for i in range(L):
            if int(row['history_rating'][i]) == 1:
                preference.append(movie_id_to_name[int(row['history_movie_id'][i])])
            else:
                unpreference.append(movie_id_to_name[int(row['history_movie_id'][i])])
        target_movie = movie_id_to_name[int(row['movie_id'])]
        preference_str = ""
        unpreference_str = ""
        for i in range(len(preference)):
            if i == 0:
                preference_str += "\"" + preference[i] + "\""
            else:
                preference_str += ", \"" + preference[i] + "\""
        for i in range(len(unpreference)):
            if i == 0:
                unpreference_str += "\"" + unpreference[i] + "\""
            else:
                unpreference_str += ", \"" + unpreference[i] + "\""
        target_preference = int(row['rating'])
        target_movie_str = "\"" + target_movie + "\""
        target_preference_str = "Yes." if target_preference == 1 else "No."
        json_list.append({
            "instruction": "Given the user's preference and unpreference, identify whether the user will like the target movie by answering \"Yes.\" or \"No.\".",
            "input": f"User Preference: {preference_str}\nUser Unpreference: {unpreference_str}\nWhether the user will like the target movie {target_movie_str}?",
            "output": target_preference_str,
        })
        
    with open(output_path, 'w') as f:
        json.dump(json_list, f, indent=4)

for j in range(20):
    csv_to_json(f'./train{j}.csv', f'./train{j}.json')
    csv_to_json(f'./val{j}.csv', f'./val{j}.json')
    csv_to_json(f'./test{j}.csv', f'./test{j}.json')




Unnamed: 0                                                        8999
user_id                                                           5956
history_timestamp    [975797944, 975797944, 975798016, 975798016, 9...
history_movie_id     ['999', '464', '288', '344', '1758', '1385', '...
history_rating                          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
movie_id                                                          3952
rating                                                               1
timestamp                                                    975798066
dtype: object
Unnamed: 0                                                        9999
user_id                                                           4657
history_timestamp    [975807758, 975807758, 975807796, 975807796, 9...
history_movie_id     ['999', '1846', '1114', '2682', '1372', '163',...
history_rating                          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
movie_id                                                       

KeyboardInterrupt: 